In [1]:
import pandas as pd
import csv
import random
import re

### 1. Verilerin Yüklenmesi

7000 veriden oluşan ve hastalık tahmininde kullandığımız veri kombinasyon dosyasını yüklüyoruz. Bu dosya değişik semptom kombinasyonlarından oluşup bu notebook'ta rasa için uygun eğitim verisi oluşturmakta kullanılacaktır.

Chatbot'un zeki olarak nitelendirilebilmesi için cümlede çeşitli varlıkları (entity) tespit edebilmesi gerekmektedir. 

Örneğin benim bugün başım ağrıyor, ateşim var, boğazım şişti ve burnum akıyor cümlesinde RASA'nın her bir semptomu (baş ağrısı, ateş, şiş boğaz ve burun akıntısı) tespit edebilmesi gerekmektedir. 

Eğitim verisi formatı şu şekildedir:

```
Benim bugün [başım ağrıyor] (semptom), [ateşim] (semptom) var, [boğazım şişti] (semptom) ve [bulanık görüyorum] (semptom)
```

Ayrıca eğitim verisi içinde görüşün netliğini belirtirken yalnızca bulanık görüyorum değil görüşüm net değil gibi aynı anlamı taşıyan ama farklı şekilde söylenmiş semptomlar da bulunmalıdır.

In [2]:
veri_komb = pd.read_csv('veri_komb.csv', encoding='iso-8859-9', sep=",")
veri_komb.set_index('hastalik').head()

,baş ağrısı,bulanık görme,burun kanaması,düzensiz kalp ritmi,göğüs ağrısı,göğüste baskı,idrarda kan,nefes darlığı,ciltte kırmızlık,kaşıntı,...,öğün atlamak,şiş boğaz,hafıza kaybı,ruh hali değişimleri,kanlı balgam,ağız içinde yumru,ağızda uyuşma,gevşek diş,iyileşmeyen ağız yarası,koyu cilt
hastalik,,,,,,,,,,,,,,,,,,,,,
addison_hastalığı,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
addison_hastalığı,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
addison_hastalığı,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
addison_hastalığı,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
addison_hastalığı,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## 2. Rasa için semptom girdi örneklerinin oluşturulması

### 2.1 Synonym oluşturulması

Kişilerin yaşadıkları semptomları çok farklı şekillerde ifade edebilirler.

Örneğin zayıfladıklarını aşağıdaki şekillerde ifade edebilirler.
- zayıfladım
- kilom azaldı
- kilomda azalış oldu
- semptomlarım arasında kilo azalışı var
- semptomlarım zayıflama

Ya da konuşmakta güçlük çektikilerini aşağıdaki gibi ifade edebilirler.
- konuşmakta güçlük çekiyorum
- konuşmakta zorlanıyorum
- dil becerilerimde kayıp var

Dilin bu özelliğini RASA'da synonym mapper ile telafi edebiliyoruz. Synonym mapper aşağıdaki formatta olup içerdiği veriler RASA'nın nlu.yml dosyasında bulunmaktadır. Biz de bu formata uygun semptom ve benzer söylenişlerini düzenleyip synonyms.txt dosyasına kaydediyoruz.  

- synonym: nefes darlığı

  examples: |
    - nefesim daralıyor
    - solunum sıkıntısı
    - nefes almakta güçlük
    - nefes almakta zorluk

In [3]:
semptomlar = pd.read_csv('semptomlar.csv', encoding='iso-8859-9', sep=";")
semptomlar.head()

,semptom,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,açık renkli dışkı,dışkımın rengi açık,dışkım açık renkte,açık renkli bir dışkı,NaN,NaN,NaN,NaN
1,açlık,aç,açlığım,NaN,NaN,NaN,NaN,NaN
2,açlık reddi,açlığımı reddediyorum,açlık hissetmiyorum,kendimi aç hissetmiyorum,NaN,NaN,NaN,NaN
3,ağız içinde yumru,ağzımda yumru,ağzımda kitle,ağzımda sert kitle,ağzımda sert doku,ağzımın içinde kitle,ağzımın içinde yumru,NaN
4,ağız kokusu,ağzım kokuyor,ağzımda koku var,ağzım kötü kokuyor,ağzımda kötü bir koku var,ağzımda kötü koku var,NaN,NaN


In [4]:
my_dict = {}

with open('semptomlar.csv', newline='',  encoding='iso-8859-9') as f_input:
    csv_input = csv.reader(f_input, delimiter=';')
    header = next(csv_input)   # skip over the header row

    for row in csv_input:
        my_dict[row[0]] = [cell for cell in row[1:] if cell]

print(my_dict)   

{'açık renkli dışkı': ['dışkımın rengi açık', 'dışkım açık renkte', 'açık renkli bir dışkı'], 'açlık': ['aç', 'açlığım'], 'açlık reddi': ['açlığımı reddediyorum', 'açlık hissetmiyorum', 'kendimi aç hissetmiyorum'], 'ağız içinde yumru': ['ağzımda yumru', 'ağzımda kitle', 'ağzımda sert kitle', 'ağzımda sert doku', 'ağzımın içinde kitle', 'ağzımın içinde yumru'], 'ağız kokusu': ['ağzım kokuyor', 'ağzımda koku var', 'ağzım kötü kokuyor', 'ağzımda kötü bir koku var', 'ağzımda kötü koku var'], 'ağızda aseton kokusu': ['ağzımda aseton kokusu', 'ağzımda aseton tadı'], 'ağızda köpük': ['ağzımdan köpük', 'ağzımın içinden köpük'], 'ağızda uyuşma': ['ağzım uyuşuyor', 'ağzımda uyuşukluk'], 'ağrısız yara': ['yaralarım ağrımıyor', 'yaralarımda ağrı yok', 'yaralarımda ağrıyor olmuyor'], 'akciğer sıvısı': ['akciğerimde sıvı', 'akciğerde sıvı'], 'akne': ['sivilce', 'sivilceler'], 'alt karın ağrısı': ['karnımın alt bölgesi ağrıyor', 'karnımın alt bölgesinde ağrı'], 'artan gözyaşı': ['gözyaşımda artış', '

In [5]:
with open("synonyms.txt", "a") as f:
  for x, y in my_dict.items():
    print(f"- synonym: {x}" + "\n" + "  " + "examples: |", file=f)
    for i in y:
      print(f"    " + "- " + i, file=f)
    print(file=f)

### 2.2 Hastanın semptom örneği girdisinin oluşturulması

In [6]:
semptom_sayisi = [1, 2, 3, 4]
cumle_baslangici = [
    "Benim",
    "Kötü hissediyorum. Benim",
    "Çok kötü bir",
    "Semptomlarım",
    "Son birkaç gündür benim",
    "Oğlum son birkaç gündür" ,
    "Dün çok kötü bir",
    "Kızım çok kötü hissediyor. Kızımın",
    "Çocuğum iyi hissetmiyor. Onun",
    "İyi hissetmiyorum. Benim",
    "Çok kötüyüm."
]

In [20]:
for sayi in semptom_sayisi:
    
    for ornek in range(1, 800):
      
      with open("semptom_ornek.txt", "a") as f:
    
        baslangic = random.choice(cumle_baslangici)
        semptomlar = []

        for i in range(len(semptom_sayisi)):

          # collect some random symtpoms
          random_list = []
          random_symptom = random.choice(list(my_dict.keys()))
          # append symptom 
          random_list.append(random_symptom) 
          # or append synoym of that symptom
          random_list.append(random.choice(my_dict[random_symptom]))

          semptom = random.choice(random_list)
          semptomlar.append(semptom)

        semptom_rasa = []
        
        for semptom in semptomlar:
            semptom = f"[{semptom}](semptom)"
            semptom_rasa.append(semptom)
            
        semptom_1 = semptom_rasa[0]
        semptom_2 = semptom_rasa[1]
        semptom_3 = semptom_rasa[2]
        semptom_4 = semptom_rasa[3]

        if sayi == 1:
            
            ornek_veri = f"- {baslangic} {semptom_1}"
            
        if sayi == 2:
            
            ornek_veri = f"- {baslangic} {semptom_1} ve {semptom_2}"
            
            
        if sayi == 3:
            
            ornek_veri = f"- {baslangic} {semptom_1}, {semptom_2}, ve {semptom_3}"
            
            
        if sayi == 4:
            
            ornek_veri = f"- {baslangic} {semptom_1}, {semptom_2}, {semptom_3}, {semptom_4}"

        print(ornek_veri, file=f)
            

### 2.3 Rasa için hastalık sorgu örneklerinin oluşturulması

In [16]:
hastalik_isimleri = pd.read_csv('hastalik_semptom.csv', sep=";")
hastaliklar = list(hastalik_isimleri['hastalik_ismi'])

In [17]:
cumle_sonlari = [
    "hakkında bilgi almak istiyorum",
    "nedir",
    "için bilgi alabilir miyim",
    "hakkında sorularım var",
    "hakkında acaba bana bilgi verebilir misiniz" ,
    "hakkında size danışmak istedim",
    ""
]

In [19]:
for ornek in range(1, 700):
  with open("hastalik_sorgu_ornek.txt", "a") as f:
    hastalik = f"[{random.choice(hastaliklar)}](hastalik)"
    cumle_sonu = random.choice(cumle_sonlari)
    ornek_veri = f"- {hastalik} {cumle_sonu}"
    print(ornek_veri, file=f)

### Ek: Semptom isimlerinin elde edilmesi

Synonym mapper yapabilmek için her semptomun olduğu bir dosyaya ihtiyacımız vardı. 

In [11]:
semptomlar_df = pd.DataFrame(veri_komb.columns, columns=["semptom"])

In [12]:
semptomlar_df.to_csv('semptomlar.csv', encoding='ISO-8859-9', index=False)